# Creating the initial position file
 #### The following code read the pourpoints extracted from Arc gis in a way that.so in each watershed we have one point. 
 Then using Pylag create initial position file it will create a release zone for each pour point/groupid, in the end each group_id will contain sets of release zone 
 6/3/2024

#### Part 1-1: import required libraries

In [1]:
import os
import pandas as pd
from pylag.processing.coordinate import get_epsg_code, utm_from_lonlat, lonlat_from_utm
from pylag.processing.coordinate import utm_from_lonlat, lonlat_from_utm
from pylag.processing.release_zone import create_release_zones_along_cord
from pylag.processing.plot import create_figure, colourmap
from pylag.processing.plot import FVCOMPlotter
from pylag.processing.release_zone import create_release_zone
from pylag.processing.input import create_initial_positions_file_multi_group
from pylag.processing.input import create_initial_positions_file_single_group

#### Part1-2: read the initial position file and rename the columns

In [2]:
# define the path to the input file
data_dir ='/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/input/initial_position'
init_file = os.path.join(data_dir, 'WatershedPourPoints_LH_copy_multigroup.dat')

cwd = os.getcwd()
# Create run directory
simulation_dir = '/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron'.format(cwd)
try:
    os.makedirs(simulation_dir)
except FileExistsError:
    pass

df = pd.read_csv(init_file, skiprows=0,sep=' ',header=None)

df.rename(columns={0:"group_id", 1:"lon",2:"lat",3:"depth"}, inplace=True)
df

,group_id,lon,lat,depth
0,0,275.334159,45.745061,0.0
1,1,275.171259,45.749943,0.0
2,2,275.513964,45.663177,0.0
3,3,275.765642,45.637350,0.0
4,4,275.535185,45.655921,0.0
...,...,...,...,...
127,128,276.574032,45.061532,0.0
128,129,275.921626,45.487312,0.0
129,130,276.603438,43.818410,0.0
130,135,277.593057,43.030980,0.0


#### Part2: Create a release zone and save it 

In [3]:
# Release zone  
radius = 0.01
# target number of particles
n_particles_target = 100

# Release depths
depth_below_surface = 0.0
# list to accumulate release zones
release_zones = []

# Loop through the rows of the DataFrame
for index, row in df.iterrows():
    group_id = row['group_id']
    lat = row['lat']
    lon = row['lon']
    
    # Create the release zone
    surface_release_zone = create_release_zone(group_id = group_id,
                                           radius = radius,
                                           centre = [lon, lat],
                                           n_particles = n_particles_target,
                                           depth = depth_below_surface,
                                           random = False)
    
    # accumulate the release zones
    release_zones.append(surface_release_zone)
    
    
 # Create input sub-directory
input_dir = os.path.join(simulation_dir, 'input/initial_position')
os.makedirs(input_dir, exist_ok=True)

    # Output filename
file_name = os.path.join(input_dir, 'WatershedPourPoints_LH_multigroup_2.dat')

# Write data to file
create_initial_positions_file_multi_group(file_name, release_zones)


# Solve an error related to PyLag

My Lat and lon number has more than 10 digits and when I run it through Pylag it gives me this error:ValueError: invalid literal for int() with base 10: '0.0' so to avoid this i want to read Lat and Lons and round them by 


In [4]:
# read file_name using pandas
from decimal import Decimal
df = pd.read_csv(file_name, skiprows=1,sep=' ',header=None)
df.rename(columns={0:"group_id", 1:"lon",2:"lat",3:"depth"}, inplace=True)
# if df[lon] and df[lat] are greater than 10 digits round them to 10 digits
df['lon'] = df['lon'].round(10)
df['lat'] = df['lat'].round(10)
df['group_id'] = df['group_id'].astype(int)
# remove the header and write the data to the file name called WatershedPourPoints_LH_multigroup_rounded.dat
df

,group_id,lon,lat,depth
0,0,275.334159,45.735061,0.0
1,0,275.328159,45.737061,0.0
2,0,275.330159,45.737061,0.0
3,0,275.332159,45.737061,0.0
4,0,275.334159,45.737061,0.0
...,...,...,...,...
10687,147,276.534212,43.746634,0.0
10688,147,276.536212,43.746634,0.0
10689,147,276.538212,43.746634,0.0
10690,147,276.540212,43.746634,0.0


In [5]:
# save the pandas dataframe to the file name called WatershedPourPoints_LH_multigroup_rounded.dat

file_name_new = os.path.join(input_dir, 'WatershedPourPoints_LH_multigroup_rounded.dat')
df.to_csv(file_name_new, sep=' ', index=False, header=False)